In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/29 21:03:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.registerTempTable('green')

In [18]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [26]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [20]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [22]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [27]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [46]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [47]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [48]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [50]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [51]:
df_join = spark.read.parquet('data/report/revenue/total')

In [56]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [54]:
df_zones = spark.read.parquet('zones/')

In [57]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [62]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

# Homework 

In [1]:
!ls

03_test.ipynb		   download_data.sh
04_pyspark.ipynb	   fhv
05_taxi_schema.ipynb	   fhv_tripdata_2019-10.csv
06_spark_sql.ipynb	   fhvhv
06_spark_sql.py		   fhvhv_tripdata_2021-01.csv
06_spark_sql_big_query.py  fhvhv_tripdata_2021-01.parquet
07_groupby_join.ipynb	   head.csv
08_rdds.ipynb		   homework.ipynb
09_spark_gcs.ipynb	   taxi+_zone_lookup.csv
cloud.md		   zones


In [7]:
df_zone_lookup = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [8]:
df_zone_lookup

DataFrame[LocationID: string, Borough: string, Zone: string, service_zone: string]

In [9]:
from pyspark.sql import types

In [10]:
schema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True),
])

In [11]:
df_zone_lookup = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('taxi+_zone_lookup.csv')

In [12]:
df_zone_lookup.schema

StructType([StructField('LocationID', IntegerType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [14]:
df_zone_lookup.write.parquet('zones', mode='overwrite')

In [17]:
df_zone_lookup_pq = spark.read.parquet('zones')
df_zone_lookup_pq.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [22]:
df_zone_lookup_pq.registerTempTable('zone_lookup_data')

In [18]:
df_fhv = spark.read.parquet('fhv/2019/10')

In [19]:
df_fhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [20]:
df_fhv = df_fhv.withColumnRenamed('dropOff_datetime', 'dropoff_datetime')

In [21]:
df_fhv.registerTempTable('fhv_data')

/home/rcata/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [26]:
joined_data = df_fhv.join(df_zone_lookup_pq, df_fhv["PUlocationID"] == df_zone_lookup_pq["LocationID"], "left")


In [28]:
joined_data.groupBy("Zone").count().show()

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|           Homecrest| 1295|
|Governor's Island...|    2|
|              Corona| 7175|
|    Bensonhurst West| 1880|
|         Westerleigh| 1317|
|      Newark Airport| 2532|
|Charleston/Totten...| 2533|
|          Douglaston|  291|
|East Concourse/Co...| 3294|
|          Mount Hope| 3973|
|      Pelham Parkway| 1522|
|         Marble Hill|  116|
|           Rego Park| 1697|
|Heartland Village...| 1054|
|Upper East Side S...| 2358|
|       Dyker Heights|  690|
|   Kew Gardens Hills|  906|
|             Bayside| 2547|
|     Jackson Heights|10952|
|TriBeCa/Civic Center| 1918|
+--------------------+-----+
only showing top 20 rows



In [31]:
pickup_counts.orderBy("count").show()

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|         Jamaica Bay|    1|
|Governor's Island...|    2|
| Green-Wood Cemetery|    5|
|       Broad Channel|    8|
|     Highbridge Park|   14|
|        Battery Park|   15|
|Saint Michaels Ce...|   23|
|Breezy Point/Fort...|   25|
|Marine Park/Floyd...|   26|
|        Astoria Park|   29|
|    Inwood Hill Park|   39|
|       Willets Point|   47|
|Forest Park/Highl...|   53|
|  Brooklyn Navy Yard|   57|
|        Crotona Park|   62|
|        Country Club|   77|
|     Freshkills Park|   89|
|       Prospect Park|   98|
|     Columbia Street|  105|
|  South Williamsburg|  110|
+--------------------+-----+
only showing top 20 rows



In [32]:
# Join FHV data with zone lookup data
joined_data = df_fhv.join(df_zone_lookup_pq, df_fhv["PUlocationID"] == df_zone_lookup_pq["LocationID"], "left")

# Group by zone and count pickups
pickup_counts = joined_data.groupBy("Zone").count()

# Find the zone with the least frequent pickups
least_frequent_zone = pickup_counts.orderBy("count").first()["Zone"]

print("Name of the least frequent pickup location zone:", least_frequent_zone)

Name of the least frequent pickup location zone: Jamaica Bay
